<a href="https://colab.research.google.com/github/CarlosIvars/APR/blob/main/uc_merced_examen_4CO11_turno2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen de la práctica 2 de APR, grupo 4CO11, 10 de enero de 2024
# Turno 2: de 13.45h a 14.30h

## El conjunto de datos uc_merced

UC Merced es un conjunto de datos de imágenes de teledetección de uso del suelo con 21 clases, con 100 imágenes por clase. Las imágenes se extrajeron manualmente de imágenes grandes de la colección de imágenes de áreas urbanas del mapa nacional de USGS para varias áreas urbanas de EEUU. La mayoría de las imágenes tienen 256x256 píxeles, pero hay 44 imágenes con diferentes dimensiones.

El conjunto de datos contiene un único conjunto, que se considera de entrenamiento, pero lo particionamos para definir un nuevo conjunto de entrenamiento (80%), un conjunto de validación (10%) y un conjunto de test (10%). A continuación se carga la partición así definida mediante la librería tensorflow_datasets.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, val_data, test_data = tfds.load('uc_merced', split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2100 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/uc_merced/2.0.0.incompleteYFY4GH/uc_merced-train.tfrecord*...:   0%|      …

Dataset uc_merced downloaded and prepared to /root/tensorflow_datasets/uc_merced/2.0.0. Subsequent calls will reuse this data.


## Ejercicio

Como hicimos en las sesiones 3 y 4 de la práctica, haz un fine-tuning (ajuste fino) de una red Keras pre-entrenada con ImageNet que clasifique el test de uc_merced con la máxima precisión posible, preferiblemente superior al 90%.

In [4]:
from keras.applications.efficientnet import preprocess_input

img_size = (299, 299)
num_classes = 21

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

In [5]:
train_data = train_data.map(preprocess)
val_data = val_data.map(preprocess)
test_data = test_data.map(preprocess)

In [6]:
print(len(train_data),len(val_data),len(test_data))

1680 210 210


In [8]:
from keras.applications.efficientnet import EfficientNetB4
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomRotation, RandomTranslation, RandomZoom
from keras.models import Model

input_layer = Input(shape=img_size + (3,))

x = RandomRotation(factor=0.1, fill_mode='nearest')(input_layer)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='nearest')(x)
x = RandomZoom(height_factor=0.2, fill_mode='nearest')(x)

efficientNet_B4_model = EfficientNetB4(input_shape=img_size + (3,),include_top=False, weights='imagenet')

for layer in efficientNet_B4_model.layers:
    layer.trainable = False

x = efficientNet_B4_model(x)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

aug_model = Model(inputs=input_layer, outputs=output)

aug_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_rotation_1 (RandomR  (None, 299, 299, 3)       0         
 otation)                                                        
                                                                 
 random_translation_1 (Rand  (None, 299, 299, 3)       0         
 omTranslation)                                                  
                                                                 
 random_zoom_1 (RandomZoom)  (None, 299, 299, 3)       0         
                                                                 
 efficientnetb4 (Functional  (None, 10, 10, 1792)      17673823  
 )                                                               
                                                           

In [9]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
aug_model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [10]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=20
batch_size=32
train_dataset_batched = train_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = aug_model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/20
53/53 [==============================] - ETA: 0s - loss: 1.1273 - accuracy: 0.6786
Epoch 1: val_accuracy improved from -inf to 0.87619, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


53/53 [==============================] - 46s 556ms/step - loss: 1.1273 - accuracy: 0.6786 - val_loss: 0.3492 - val_accuracy: 0.8762 - lr: 0.0010
Epoch 2/20
53/53 [==============================] - ETA: 0s - loss: 0.4108 - accuracy: 0.8696
Epoch 2: val_accuracy improved from 0.87619 to 0.89524, saving model to best_model.h5
53/53 [==============================] - 17s 327ms/step - loss: 0.4108 - accuracy: 0.8696 - val_loss: 0.2785 - val_accuracy: 0.8952 - lr: 0.0010
Epoch 3/20
53/53 [==============================] - ETA: 0s - loss: 0.3337 - accuracy: 0.8935
Epoch 3: val_accuracy improved from 0.89524 to 0.91429, saving model to best_model.h5
53/53 [==============================] - 18s 340ms/step - loss: 0.3337 - accuracy: 0.8935 - val_loss: 0.2373 - val_accuracy: 0.9143 - lr: 0.0010
Epoch 4/20
53/53 [==============================] - ETA: 0s - loss: 0.2559 - accuracy: 0.9113
Epoch 4: val_accuracy did not improve from 0.91429
53/53 [==============================] - 16s 310ms/step - lo

In [11]:
aug_model = load_model('best_model.h5')
test_dataset_batched = test_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = aug_model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 24.79
Test accuracy: 92.38


He optado por usar EfficientNetB4, el cual obtiene muy buenos resulados, cuenta con 12.3M de parametros y su tiempo de ejecucion en gpu es de 8.8 ms


Podemos observar que nos da un resultado de 92.38% de acierto: